In [1]:
# Put the correct credentials here and mount S3 bucket
ACCESS_KEY = "ACCESS_KEY"
SECRET_KEY = "SECRET_KEY/RwmlRJ"
AWS_BUCKET_NAME = "AWS_BUCKET_NAME"
MOUNT_NAME = "MOUNT_NAME"

ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")

try: 
  dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
except:
  pass

In [2]:
# For now we will just read data from a text file
input_rdd = sc.textFile("/mnt/%s/flights.csv" % MOUNT_NAME)
input_list_rdd = input_rdd.filter(lambda line: 'YEAR' not in line).map(lambda line: line.split(','))

In [3]:
# A function to coerce string to integer
# note: if value is negative we return a 0
def return_delay(delay):
    try: delay = int(delay)
    except: delay = 0
      
    return delay if delay > 0 else 0

In [4]:
 delay_rdd = input_list_rdd.map(lambda x: (return_delay(x[11]), return_delay(x[22])))

In [5]:
delay_rdd.first()

### Actions
Actions are RDD methods that return a value to a driver program. This section discusses the commonly used
RDD actions.

##### collect
The collect method returns the elements in the source RDD as an array. This method should be used with
caution since it moves data from all the worker nodes to the driver program. It can crash the driver program
if called on a very large RDD.

In [8]:
input_list_rdd.map(lambda x: x[4]).distinct().collect()

##### count
The count method returns a count of the elements in the source RDD

In [10]:
input_list_rdd.count()

##### countByValue
The countByValue method returns a count of each unique element in the source RDD. It returns an instance
of the Map class containing each unique element and its count as a key-value pair.

In [12]:
input_list_rdd.filter(lambda x: 'AIRLINE' not in x).map(lambda x: x[4]).countByValue()

##### first
The first method returns the first element in the source RDD

In [14]:
input_rdd.first()

##### max
The max method returns the largest element in an RDD

In [16]:
delay_rdd.flatMap(lambda x: x).max()

##### min
The min method returns the smallest element in an RDD.

In [18]:
delay_rdd.flatMap(lambda x: x).min()

##### take
The take method takes an integer N as input and returns an array containing the first N element in the
source RDD.

In [20]:
delay_rdd.take(10)

##### takeOrdered
The takeOrdered method takes an integer N as input and returns an array containing the N smallest
elements in the source RDD

In [22]:
delay_rdd.flatMap(lambda x: x).takeOrdered(3)

In [23]:
delay_rdd.flatMap(lambda x: x).takeOrdered(5, key = lambda x: -x)

##### top
The top method takes an integer N as input and returns an array containing the N largest elements in the
source RDD.

In [25]:
delay_rdd.flatMap(lambda x: x).top(3)

### Actions on RDD of Numeric Types

RDDs containing data elements of type Integer, Long, Float, or Double support a few additional actions that are useful for statistical analysis. The commonly used actions from this group are briefly described next

##### mean
The mean method returns the average of the elements in the source RDD.

In [28]:
delay_rdd.flatMap(lambda x: x).mean()

##### stdev
The stdev method returns the standard deviation of the elements in the source RDD.

In [30]:
delay_rdd.flatMap(lambda x: x).stdev()

##### sum
The sum method returns the sum of the elements in the source RDD

In [32]:
delay_rdd.flatMap(lambda x: x).sum()

#### variance
The variance method returns the variance of the elements in the source RDD

In [34]:
delay_rdd.flatMap(lambda x: x).sum()

### Saving an RDD

##### saveAsTextFile
The saveAsTextFile method saves the elements of the source RDD in the specified directory on any
Hadoop-supported file system. Each RDD element is converted to its string representation and stored as a
line of text

##### saveAsObjectFile
The saveAsObjectFile method saves the elements of the source RDD as serialized Java objects in the specified directory.

##### saveAsSequenceFile
The saveAsSequenceFile method saves an RDD of key-value pairs in SequenceFile format. An RDD of keyvalue pairs can also be saved in text format using the saveAsTextFile.